In [28]:
import sys
sys.path.append('..')
sys.path.append('../..')
import os
from utils import *
from student_utils_sp18 import *
from networkx.algorithms.traversal import *
from networkx.algorithms.approximation import *
from vasilis_solver import *
from simple_solver import *
from new_method_solver import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [54]:
version1 = 0
version2 = 0
version3 = 0


def ensemble(G, start_index):
    
    global version1, version2, version3
    
    # solver 1
    t1, c1 = simple(G, start_index)
    
    score1, text1 = cost_of_solution(G, t1, c1)
    total_cost1 = re.search('\d+(\.\d{1,100000})?', text1.split("\n")[2]).group(0)
    
    # solver 2
    t2, c2 = vasilis_solver(G, start_index)
    
    score2, text2 = cost_of_solution(G, t2, c2)
    total_cost2 = re.search('\d+(\.\d{1,100000})?', text2.split("\n")[2]).group(0)
         
    # solver 3
    t3, c3 = new_method(G, start_index)
    
    score3, text3 = cost_of_solution(G, t3, c3)
    total_cost3 = re.search('\d+(\.\d{1,100000})?', text3.split("\n")[2]).group(0)
    
    mini = min(score1, score2, score3)
    
    if score1==mini:
        version1+=1
        return t1, c1
    elif score2==mini:
        version2+=1
        return t2, c2
    elif score3==mini:
        version3+=1
        return t3, c3
    else:
        print("We are funcked")

In [55]:
ensemble_3_solution = solve_all(ensemble)


  6%|▌         | 42/716 [01:13<19:45,  1.76s/it]

pipes


 15%|█▍        | 107/716 [02:58<16:55,  1.67s/it]

pipes


 17%|█▋        | 123/716 [03:25<16:30,  1.67s/it]

...

 94%|█████████▍| 676/716 [22:26<01:19,  1.99s/it]

pipes


 95%|█████████▍| 677/716 [22:27<01:17,  1.99s/it]

pipes


100%|██████████| 716/716 [23:59<00:00,  2.01s/it]


In [56]:
total_scores = []
conquering_cost = []
travelling_cost = []
filename = []

for k in ensemble_3_solution:
    total_scores.append(float(ensemble_3_solution[k]['total_cost']))
    conquering_cost.append(float(ensemble_3_solution[k]['conquering_cost']))
    travelling_cost.append(float(ensemble_3_solution[k]['travelling_cost']))
    filename.append(k)
    
import pandas as pd

ensemble_3_solution_result = pd.DataFrame(data = {'filename':filename, 'total_scores':total_scores, 'conquering_cost':conquering_cost, 
                     'travelling_cost':travelling_cost})
ensemble_3_solution_result.head()

,conquering_cost,filename,total_scores,travelling_cost
0,67418.0,inputs/234.in,1.012580e+05,3.384000e+04
1,866024.0,inputs/374.in,5.723158e+07,5.636556e+07
2,2600.0,inputs/265.in,4.576000e+03,1.976000e+03
3,170.0,inputs/325.in,1.078000e+03,9.080000e+02
4,5.0,inputs/296.in,2.100000e+01,1.600000e+01


In [57]:
past_result = pd.read_csv("results_log/vasilis_greedy.csv")

In [58]:
s = version1 + version2 + version3
version1/s, version2/s, version3/s

(0.4329608938547486, 0.39804469273743015, 0.16899441340782123)

In [66]:
ix = ensemble_3_solution_result['total_scores']<past_result['total_scores'].values

a = (past_result.loc[ix, :]['total_scores'] - ensemble_3_solution_result.loc[ix, :]['total_scores'])/past_result.loc[ix, :]['total_scores']
np.mean(a)

0.2657102034576518

In [2]:
def solve_all(solver):
    indirectory = "inputs/"
    outdirectory = "outputs/"
    
    #os.makedirs(outdirectory)
    
    infiles = os.listdir(indirectory)
    
    solution_data = {}
    
    for input_file in tqdm(infiles):
            input_data = read_file(indirectory+input_file)
            number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = data_parser(input_data)
            
            G, start_index = get_G(number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix)
            try: 
                closed_walk, conquered_kingdoms = solver(G, start_index)
            except KeyError as e:
                print(e)
                print("Error in input file: {}".format(input_file))
            
            
            # get the score of our solution
            score, text = cost_of_solution(G, closed_walk, conquered_kingdoms)
            
            try:
                conquering_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[0]).group(0)
                travelling_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[1]).group(0)
                total_cost = re.search('\d+(\.\d{1,100000})?', text.split("\n")[2]).group(0)
                
                solution_data[indirectory+input_file] = {"conquering_cost":conquering_cost, "travelling_cost":travelling_cost,
                                                    "total_cost":total_cost}
            except AttributeError:
                print(indirectory+input_file)
                print(text)
       
            closed_walk = [list_of_kingdom_names[i] for i in closed_walk]
            conquered_kingdoms = [list_of_kingdom_names[i] for i in conquered_kingdoms]
            

            output_file = outdirectory + input_file[:-3] +".out"
            write_data_to_file(output_file, closed_walk, ' ')
            write_to_file(output_file, '\n', append=True)
            write_data_to_file(output_file, conquered_kingdoms, ' ', append=True)
            
    return solution_data

In [6]:
def get_G(number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix):
    
    name2index = {}
    index2name = {}

    count = 0
    for name in list_of_kingdom_names:
        name2index[name] = count
        index2name[count] = name
        count += 1

    G = adjacency_matrix_to_graph(adjacency_matrix)
    
    color = {}

    for node in G.nodes():
            color[node] = "white"


    conquering_cost = {}
    for node in G.nodes():
        conquering_cost[node] = adjacency_matrix[node][node]

    nx.set_node_attributes(G, conquering_cost, "conquering_cost")
    nx.set_node_attributes(G, color, "color")
    
    return G, name2index[starting_kingdom]